# L5: Long-context understanding

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Load API keys and libraries

In [ ]:
import os
from utils import get_llama_api_key, get_llama_base_url, get_together_api_key
from utils import get_hf_access_token, get_github_access_token

llama_api_key = get_llama_api_key()
llama_base_url = get_llama_base_url()
together_api_key = get_together_api_key()
hf_access_token = get_hf_access_token()
github_access_token = get_github_access_token()

from utils import llama4, llama4_together
from transformers import AutoTokenizer

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

In [ ]:
model_id = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
prompt = "This is a test prompt."
num_tokens = len(tokenizer.encode(prompt))
print(f"Number of tokens: {num_tokens}")

## Large book

In [ ]:
with open("files/war-and-peace.txt", "r", encoding='utf=8') as file:
    wp = file.read()
len(wp), len(tokenizer.encode(wp))

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [ ]:
print(llama4_together(f"give me a summary of the book below: {wp}"))

## Multiple document summarization

In [ ]:
from utils import pdf2text

papers = ["2407.16833.pdf", "2409.01666.pdf", "2411.03538.pdf", "2503.17407.pdf", "2404.06654.pdf"]
paper_texts = []
for n, paper in enumerate(papers):
  text = pdf2text(f"files/{paper}")
  paper_texts.append(f"Paper {n+1}:\n{text}")
  print(paper, len(text), len(tokenizer.encode(text)))

total_text = "\n\n".join(paper_texts)
print(f"""Total papers: {len(total_text)},
{len(tokenizer.encode(total_text))}""")

In [ ]:
print(llama4_together(f"""give me a summary of the five papers
below: {total_text}"""))

## Single-hop codebase question answering

In [ ]:
from utils import download_and_extract_repo, get_py_files, write_files_to_text

repo_url = "https://github.com/meta-llama/llama-models"
repo_name = repo_url.rstrip('/').split('/')[-1]
extract_dir = f"./{repo_name}"

download_repo = False # The repo is already downloaded and extracted on the platform. If you wouldf like to try this for another repo, please change this to True.
if download_repo:
    download_and_extract_repo(repo_url, extract_dir)

py_files = get_py_files(extract_dir)
output_file = f"files/{repo_name}_files.txt"
write_files_to_text(py_files, output_file)

print(f"Output written to {output_file}")

In [ ]:
!head -100 files/llama-models_files.txt

In [ ]:
with open("files/llama-models_files.txt", "r", encoding='utf=8') as file:
    repo = file.read()

In [ ]:
prompt = f"""Which file in the content below, which consists of a list
of source file path and its content, has the function _encode_image
defined? content:
{repo}"""
print(llama4_together(prompt))

## Analyze extensive user activity

In [ ]:
from utils import get_pull_requests
import pickle
repo_owner = "meta-llama"
repo_name = "llama-cookbook"

# We have saved the pull_requests as a pickle file to make this run faster by loading files from local.
cache_file = "meta-llama_llama-cookbook_pull_requests.pkl"
load_cache_file = True

if load_cache_file:
    with open(cache_file, "rb") as f:
        pull_requests = pickle.load(f)
else:
    pull_requests = get_pull_requests(repo_owner, repo_name)
    with open(cache_file, "wb") as f:
        pickle.dump(pull_requests, f)

for pr in pull_requests:
    print(pr["number"], pr["title"])

In [ ]:
import pickle

pickle.dump(pull_requests, open(f"{repo_owner}_{repo_name}_pull_requests.pkl", "wb"))

In [ ]:
len(pull_requests)

In [ ]:
pull_requests[0]

In [ ]:
from utils import get_pr_content

all_content = []
for pr in pull_requests:
    pr_number = pr['number']
    content = get_pr_content(repo_owner, repo_name,
                             pr_number, github_access_token) # a github access token is needed to avoid the lower rate limit of making github requeests
    if content:
        all_content.append(f"PR #{pr_number}: {content}")

len(all_content)

In [ ]:
all_pr_content = "\n\n".join(all_content)
len(all_pr_content), len(tokenizer.encode(all_pr_content))

In [ ]:
query = "how many PRs are about android or iOS?"
print(llama4_together(f"""{query}
Below is all the PR info:
{all_pr_content}
"""))

In [ ]:
query = f"""how many PRs are about agents? give a one-sentence
summary of each, then a summary of all those agent PRs."""

print(llama4_together(f"""{query}
Below is all the PR info:
{all_pr_content}
"""))